In [98]:
import sys
import os
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import random
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
outDir = thisDir + 'output/mL/results/ridge/acc/'
import classification
subDir=thisDir+'output/subNetwork/'
randDir=thisDir+'output/randomIdx/results/ridge/acc/'
netList=['unassign','default', 'visual','fp','dan','van','salience','co','sm','sm-lat', 'auditory','pmn','pon']
feature=[14808,10824,8736, 4620,5264,3151, 484, 4060, 2375,316, 564,45,21]
netRoi=dict([('unassign',14808),('default', 10824),('visual',8736),('fp', 4620),('dan',5264),('van',3151),('salience', 484),('co', 4060),('sm', 2375),('sm-lat', 316),('auditory', 564),('pmn',45),('pon',21)])

#SS
SS=pd.read_csv(outDir+'SS/folds/allFolds.csv')
BS=pd.read_csv(outDir+'BS/folds/allFolds.csv')
#motor
SS_motor=SS[SS.train=='motor'].acc.values
BS_motor=BS[BS.train_task=='motor'].acc.values

#mixed
SS_mixed=SS[SS.train=='mixed'].acc.values
BS_mixed=BS[BS.train_task=='mixed'].acc.values

#memory
SS_mem=SS[SS.train=='mem'].acc.values
BS_mem=BS[BS.train_task=='mem'].acc.values


#DS
DS=pd.read_csv(outDir+'DS/folds/allFolds.csv')
#CV
CV=pd.read_csv(outDir+'CV/reformedFolds.csv')
#motor
DS_motor=DS[DS.task=='Motor'].acc.values
CV_motor=CV[CV.task=='Motor'].acc.values

#mixed
DS_mixed=DS[DS.task=='Mixed'].acc.values
CV_mixed=CV[CV.task=='Mixed'].acc.values

#memory
DS_mem=DS[DS.task=='Memory'].acc.values
CV_mem=CV[CV.task=='Memory'].acc.values






#Random sampling and feature selection calculate z scores
#subnetworks overlapped with random network sampling
SS_nets=pd.DataFrame()
DS_nets=pd.DataFrame()
CV_nets=pd.DataFrame()
BS_nets=pd.DataFrame()
for i in netList:
    tmpSS=pd.read_csv(subDir+i+'/results/ridge/acc/SS/acc.csv')
    #tmpSS['Analysis']='ridge'
    tmpSS['Network']=i
    tmpSS['Features']=netRoi[i]
    SS_nets=pd.concat([SS_nets,tmpSS])
    
    tmpDS=pd.read_csv(subDir+i+'/results/ridge/acc/DS/acc.csv')
    tmpDS['Network']=i
    tmpDS['Features']=netRoi[i]
    DS_nets=pd.concat([DS_nets,tmpDS])
    
    tmpBS=pd.read_csv(subDir+i+'/results/ridge/acc/BS/acc.csv')
    tmpBS['Network']=i
    tmpBS['Features']=netRoi[i]
    BS_nets=pd.concat([BS_nets,tmpBS])
    
    tmpCV=pd.read_csv(subDir+i+'/results/ridge/acc/CV/acc.csv')
    tmpCV=tmpCV.melt(id_vars='sub', value_vars=['mixed', 'motor','mem'], var_name='Task', value_name='Accuracy')
    tmpCV['Network']=i
    tmpCV['Features']=netRoi[i]
    CV_nets=pd.concat([CV_nets,tmpCV])
    
    
SS_nets.drop(columns='Unnamed: 0', inplace=True)
DS_nets.drop(columns='Unnamed: 0', inplace=True)
BS_nets.drop(columns='Unnamed: 0', inplace=True)
CV_nets.rename(columns={'Accuracy':'acc'},inplace=True)


#netRoi
SStotal=[]
DStotal=[]
CVtotal=[]
BStotal=[]
for i in netList:
    #CV
    CVtmp=CV_nets[(CV_nets['Network']==i)].acc.values
    CVacc=CVtmp.mean()
    CVtotal.append(CVacc)
    #SS
    SStmp=SS_nets[(SS_nets['Network']==i)].acc.values
    SSacc=SStmp.mean()
    SStotal.append(SSacc)
    #DS
    DStmp=DS_nets[(DS_nets['Network']==i)].acc.values
    DSacc=DStmp.mean()
    DStotal.append(DSacc)
    #BS
    BStmp=BS_nets[(BS_nets['Network']==i)].acc.values
    BSacc=BStmp.mean()
    BStotal.append(BSacc)

CV_mu=pd.DataFrame({'Network':netList, 'Acc':CVtotal,'Features':feature})
SS_mu=pd.DataFrame({'Network':netList, 'Acc':SStotal,'Features':feature})
DS_mu=pd.DataFrame({'Network':netList, 'Acc':DStotal,'Features':feature})
BS_mu=pd.DataFrame({'Network':netList, 'Acc':BStotal,'Features':feature})

#random samples
rand_CV=pd.read_csv(randDir+'CV/all.csv', usecols=[1,2,3])
rand_SS=pd.read_csv(randDir+'SS/all.csv',usecols=[1,2,3,4,5])
rand_DS=pd.read_csv(randDir+'DS/all.csv',usecols=[1,2,3,4,5])
rand_BS=pd.read_csv(randDir+'BS/all.csv',usecols=[1,2,3,4,5,6])


#zscore from the null model
rand_CV_mu=rand_CV['Accuracy'].mean()
rand_CV_sd=rand_CV['Accuracy'].std()

rand_SS_mu=rand_SS['acc'].mean()
rand_SS_sd=rand_SS['acc'].std()

rand_DS_mu=rand_DS['acc'].mean()
rand_DS_sd=rand_DS['acc'].std()

rand_BS_mu=rand_BS['acc'].mean()
rand_BS_sd=rand_BS['acc'].std()

CV_mu['z_score']=CV_mu['Acc']-rand_CV_mu/(rand_CV_sd)
SS_mu['z_score']=SS_mu['Acc']-rand_SS_mu/(rand_SS_sd)
DS_mu['z_score']=DS_mu['Acc']-rand_DS_mu/(rand_DS_sd)
BS_mu['z_score']=BS_mu['Acc']-rand_BS_mu/(rand_BS_sd)


#CV_stats=pd.read_csv(outDir+'CV/stats.csv')
#DS_stats=pd.read_csv(outDir+'DS/stats.csv')

In [79]:
startIndex=0
endIndex=560
iterations=0
cycle=560
df=pd.DataFrame()

while endIndex<56000:
    tmp=permBS_motorMem[startIndex:endIndex]
    tmp['iteration']=iterations
    startIndex=startIndex+560
    endIndex=endIndex+560
    iterations=iterations+1
    print(startIndex, endIndex, iterations)
    df=pd.concat([df,tmp])

560 1120 1
1120 1680 2
1680 2240 3
2240 2800 4
2800 3360 5
3360 3920 6
3920 4480 7
4480 5040 8
5040 5600 9
5600 6160 10
6160 6720 11
6720 7280 12
7280 7840 13
7840 8400 14
8400 8960 15
8960 9520 16
9520 10080 17
10080 10640 18
10640 11200 19
11200 11760 20
11760 12320 21
12320 12880 22
12880 13440 23
13440 14000 24
14000 14560 25
14560 15120 26
15120 15680 27
15680 16240 28
16240 16800 29
16800 17360 30
17360 17920 31
17920 18480 32
18480 19040 33
19040 19600 34
19600 20160 35
20160 20720 36
20720 21280 37
21280 21840 38
21840 22400 39
22400 22960 40
22960 23520 41
23520 24080 42
24080 24640 43
24640 25200 44
25200 25760 45
25760 26320 46
26320 26880 47
26880 27440 48
27440 28000 49
28000 28560 50
28560 29120 51
29120 29680 52
29680 30240 53
30240 30800 54
30800 31360 55
31360 31920 56
31920 32480 57
32480 33040 58
33040 33600 59
33600 34160 60
34160 34720 61
34720 35280 62
35280 35840 63
35840 36400 64
36400 36960 65
36960 37520 66
37520 38080 67
38080 38640 68
38640 39200 69
39200 39

274400 274960 490
274960 275520 491
275520 276080 492
276080 276640 493
276640 277200 494
277200 277760 495
277760 278320 496
278320 278880 497
278880 279440 498
279440 280000 499
280000 280560 500
280560 281120 501
281120 281680 502
281680 282240 503
282240 282800 504
282800 283360 505
283360 283920 506
283920 284480 507
284480 285040 508
285040 285600 509
285600 286160 510
286160 286720 511
286720 287280 512
287280 287840 513
287840 288400 514
288400 288960 515
288960 289520 516
289520 290080 517
290080 290640 518
290640 291200 519
291200 291760 520
291760 292320 521
292320 292880 522
292880 293440 523
293440 294000 524
294000 294560 525
294560 295120 526
295120 295680 527
295680 296240 528
296240 296800 529
296800 297360 530
297360 297920 531
297920 298480 532
298480 299040 533
299040 299600 534
299600 300160 535
300160 300720 536
300720 301280 537
301280 301840 538
301840 302400 539
302400 302960 540
302960 303520 541
303520 304080 542
304080 304640 543
304640 305200 544
305200 305

535360 535920 956
535920 536480 957
536480 537040 958
537040 537600 959
537600 538160 960
538160 538720 961
538720 539280 962
539280 539840 963
539840 540400 964
540400 540960 965
540960 541520 966
541520 542080 967
542080 542640 968
542640 543200 969
543200 543760 970
543760 544320 971
544320 544880 972
544880 545440 973
545440 546000 974
546000 546560 975
546560 547120 976
547120 547680 977
547680 548240 978
548240 548800 979
548800 549360 980
549360 549920 981
549920 550480 982
550480 551040 983
551040 551600 984
551600 552160 985
552160 552720 986
552720 553280 987
553280 553840 988
553840 554400 989
554400 554960 990
554960 555520 991
555520 556080 992
556080 556640 993
556640 557200 994
557200 557760 995
557760 558320 996
558320 558880 997
558880 559440 998
559440 560000 999


In [86]:
SS_mem.size

160

In [51]:
stats.ttest_ind(SS_motor, BS_motor, equal_var = False)

stats.ttest_ind(SS_mixed, BS_mixed, equal_var = False)

stats.ttest_ind(SS_mem, BS_mem, equal_var = False)

Ttest_indResult(statistic=4.378975855773864, pvalue=1.8125319013851326e-05)

In [28]:
#motor train, mem test
SS_motorMem=SS[(SS['train']=='motor') & (SS['test']=='mem')].acc.values
BS_motorMem=BS[(BS['train_task']=='motor') & (BS['test_task']=='mem')].acc.values
#motor train mix test
SS_motorMix=SS[(SS['train']=='motor') & (SS['test']=='mixed')].acc.values
BS_motorMix=BS[(BS['train_task']=='motor') & (BS['test_task']=='mixed')].acc.values

#mixed train, mem test
SS_mixMem=SS[(SS['train']=='mixed') & (SS['test']=='mem')].acc.values
BS_mixMem=BS[(BS['train_task']=='mixed') & (BS['test_task']=='mem')].acc.values
#mixed train motor test
SS_mixMotor=SS[(SS['train']=='mixed') & (SS['test']=='motor')].acc.values
BS_mixMotor=BS[(BS['train_task']=='mixed') & (BS['test_task']=='motor')].acc.values

#memory train, mix test
SS_memMix=SS[(SS['train']=='mem') & (SS['test']=='mixed')].acc.values
BS_memMix=BS[(BS['train_task']=='mem') & (BS['test_task']=='mixed')].acc.values
#memory train motor test
SS_memMotor=SS[(SS['train']=='mem') & (SS['test']=='motor')].acc.values
BS_memMotor=BS[(BS['train_task']=='mem') & (BS['test_task']=='motor')].acc.values


In [79]:
stats.ttest_ind(SS_motorMem, BS_motorMem, equal_var = False)

stats.ttest_ind(SS_motorMix, BS_motorMix, equal_var = False)

stats.ttest_ind(SS_mixMem, BS_mixMem, equal_var = False)

stats.ttest_ind(SS_mixMotor, BS_mixMotor, equal_var = False)

stats.ttest_ind(SS_memMix, BS_memMix, equal_var = False)

stats.ttest_ind(SS_memMotor, BS_memMotor, equal_var = False)

Ttest_indResult(statistic=3.826524046275118, pvalue=0.00021511136616734808)

In [17]:
stats.ttest_ind(DS_motor, CV_motor, equal_var = False)

stats.ttest_ind(DS_mixed, CV_mixed, equal_var = False)

stats.ttest_ind(DS_mem, CV_mem, equal_var = False)

Ttest_indResult(statistic=-15.852835751851044, pvalue=2.1359668634679154e-26)

In [ ]:
#permutations

In [34]:
stats.ttest_ind(perm_DS_motor, perm_CV_motor,equal_var=False)

#stats.ttest_ind(perm_DS_mixed, perm_CV_mixed)

#stats.ttest_ind(perm_DS_mem, perm_CV_mem)

Ttest_indResult(statistic=-23.409450833212784, pvalue=5.11626062187751e-118)

In [33]:
#perm_DS_mem.mean()
#perm_DS_mem.std()

#perm_DS_motor.mean()
#perm_DS_motor.std()

#perm_DS_mixed.mean()
#perm_DS_mixed.std()

#perm_CV_mem.mean()
#perm_CV_mem.std()

#perm_CV_motor.mean()
#perm_CV_motor.std()

#perm_CV_mixed.mean()
#perm_CV_mixed.std()

0.12813060719437805

In [41]:
#stats.ttest_ind(permSS_motorMem, permBS_motorMem)

#stats.ttest_ind(permSS_motorMix, permBS_motorMix)

#stats.ttest_ind(permSS_mixMem, permBS_mixMem)

#stats.ttest_ind(permSS_mixMotor, permBS_mixMotor)

#stats.ttest_ind(permSS_memMix, permBS_memMix)

#stats.ttest_ind(permSS_memMotor, permBS_memMotor)


Ttest_indResult(statistic=-0.004505608163070661, pvalue=0.9964051131511842)

In [53]:
#permSS_motorMem.mean()
#permBS_motorMem.mean()

#permSS_motorMix.mean()
#permBS_motorMix.mean()

#permSS_memMix.mean()
#permBS_memMix.mean()

#permSS_memMotor.mean()
#permBS_memMotor.mean()

#permSS_mixMem.mean()
#permBS_mixMem.mean()

#permSS_mixMotor.mean()
permBS_mixMotor.mean()




0.4990675

In [65]:
#permSS_motorMem.std()
#permBS_motorMem.std()

#permSS_motorMix.std()
#permBS_motorMix.std()

#permSS_memMix.std()
#permBS_memMix.std()

#permSS_memMotor.std()
#permBS_memMotor.std()

#permSS_mixMotor.std()
#permBS_mixMotor.std()

#permSS_mixMem.std()
permBS_mixMem.std()





0.03906521438569101

In [89]:
SS

,train,test,sub,acc
0,mixed,mem,MSC01,0.80
1,mixed,mem,MSC01,0.95
2,mixed,mem,MSC01,0.95
3,mixed,mem,MSC01,0.95
4,mixed,mem,MSC01,0.95
5,mixed,mem,MSC01,0.95
6,mixed,mem,MSC01,0.95
7,mixed,mem,MSC01,0.95
8,mixed,mem,MSC01,0.95
9,mixed,mem,MSC01,0.95


In [99]:
SS=pd.read_csv(outDir+'SS/acc.csv')
BS=pd.read_csv(outDir+'BS/acc.csv')
DS=pd.read_csv(outDir+'DS/acc.csv')

In [103]:
tmp=BS[(BS.train_task=='mem')&(BS.test_task=='motor')]
tmp.reset_index(inplace=True)

In [115]:
tmp=DS[(DS.task=='motor')]
tmp.reset_index(inplace=True)

In [116]:
tmp

,index,Unnamed: 0,train_sub,test_sub,task,acc
0,1,1,MSC01,MSC02,motor,0.925
1,4,4,MSC01,MSC03,motor,0.925
2,7,7,MSC01,MSC04,motor,0.995
3,10,10,MSC01,MSC05,motor,0.885
4,13,13,MSC01,MSC06,motor,0.760
5,16,16,MSC01,MSC07,motor,0.965
6,19,19,MSC01,MSC10,motor,0.985
7,22,22,MSC02,MSC01,motor,0.500
8,25,25,MSC02,MSC03,motor,0.725
9,28,28,MSC02,MSC04,motor,0.580
